## Cümle Analizi

**Hedefimiz girdi olarak verilen her cümlenin olumlu mu olumsuz mu olduğunu bulmak.**

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from IPython.display import clear_output

In [4]:
import numpy as np
import pandas as pd
import enum

In [5]:
from tensorflow.python.keras.models import Sequential,load_model
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

## 1- Veriseti

* Halihazırda önişlemeden geçtiği için tüm cümleler 0 veya 1 olarak işaretlenmiş durumda.
    * 0 olumsuz 
    * 1 olumlu

In [6]:
dataset = pd.read_csv('../data/data.csv')

In [7]:
print("Verisetinde {} adet cümle mevcut.".format(len(dataset)))

Verisetinde 243497 adet cümle mevcut.


Örnekleri rahatça görebileceğimiz basit bir fonksiyon yazabiliriz.

In [8]:
def see_example(sentence, rate):
    tag = "Olumlu" if rate else "Olumsuz"
    print("Cümle: {} \nEtiket: {}".format(sentence, tag))

## 2- Eğitim ve Test Ayırımı

Cümlelerin %80'ini eğitim, geri kalanını test olarak ayıralım.

In [9]:
target = dataset['Rating'].values.tolist()
data = dataset['Review'].values.tolist()

In [10]:
ratio = int(len(data) * .80)
x_train, y_train = data[:ratio], target[:ratio]
x_test, y_test   = data[ratio:], target[ratio:]

In [11]:
print("{} adet cümle eğitim için kullanılacak.".format(len(x_train)))
print("{} adet cümle test için kullanılacak.".format(len(x_test)))

194797 adet cümle eğitim için kullanılacak.
48700 adet cümle test için kullanılacak.


Birer olumlu ve olumsuz örnek bakalım.

In [12]:
IDX = 22223
see_example(sentence=x_train[IDX], rate=y_train[IDX])

Cümle: arkdaslar cok arastirma  yaptim  bütün telfonlari arastirdim bu telefonda karar kildim harika bir telefon herseyiy le mükemmel herkese öneriyorum  
Etiket: Olumlu


In [13]:
IDX = 2139
see_example(sentence=x_train[IDX], rate=y_train[IDX])

Cümle: ilk başta iş görür gibi görünüyor ama siz oyuncusunuz günde 300-400 tık yapıyorsunuz 3-4 aya tık ömrü bitiyo sol tıkı kullanamaz hale geliyorsunuz buda aklınızda bulunsun 
Etiket: Olumsuz


## 3- Önişleme

Şimdi tokenize işlemine geçip tüm kelimelere birer sayı ataması yapacağız. Ama önce kelime sayısını sınırlandırmakta fayda var. En çok kullanılan 10 bin kelimeyi ele alalım.

In [14]:
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)

10 bin kelimelik bir tokenizer oluşturduk şimdi kendi cümlelerimizin tamamını girdi olarak verip kelimelerin **entropi** değerine göre nasıl sıralandığını gözlemleyelim. 

In [15]:
tokenizer.fit_on_texts(data)
tokenizer.word_index

{'çok': 1,
 'bir': 2,
 've': 3,
 'ürün': 4,
 'bu': 5,
 'iyi': 6,
 'güzel': 7,
 'için': 8,
 'tavsiye': 9,
 'ederim': 10,
 'daha': 11,
 'ama': 12,
 'da': 13,
 'gayet': 14,
 'hızlı': 15,
 'teşekkürler': 16,
 'aldım': 17,
 'de': 18,
 'ürünü': 19,
 'gibi': 20,
 'yok': 21,
 'uygun': 22,
 'olarak': 23,
 'kaliteli': 24,
 'en': 25,
 '2': 26,
 'kargo': 27,
 'fiyat': 28,
 'elime': 29,
 'kadar': 30,
 'ile': 31,
 'göre': 32,
 'geldi': 33,
 'var': 34,
 'hepsiburada': 35,
 'ben': 36,
 'gerçekten': 37,
 '1': 38,
 'fiyata': 39,
 'gün': 40,
 'sonra': 41,
 'cok': 42,
 'kesinlikle': 43,
 'telefon': 44,
 'biraz': 45,
 'hiç': 46,
 'ulaştı': 47,
 'memnun': 48,
 'hem': 49,
 'değil': 50,
 'kullanışlı': 51,
 '3': 52,
 'mükemmel': 53,
 'oldu': 54,
 'kullanıyorum': 55,
 'önce': 56,
 'sipariş': 57,
 'tek': 58,
 'her': 59,
 'bence': 60,
 'harika': 61,
 'kalitesi': 62,
 'bi': 63,
 'ayrıca': 64,
 '5': 65,
 'teşekkür': 66,
 'fiyatı': 67,
 'olması': 68,
 'ne': 69,
 'herkese': 70,
 'bile': 71,
 'uzun': 72,
 'süper': 73,

En çok kullanılan kelimelerin **çok, bir, ve, ürün** olduklarını görebiliriz.

Şimdi cümleleri kelimeler yerine bu sayılarla oluşturalım.

In [16]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

Cümlelerimizin yeni ve eski hallerini gözlemleyelim.

In [17]:
IDX = 10
print("Öncesi: {}".format(x_train[IDX]))
print("Sonrası: {}".format(np.array(x_train_tokens[IDX])))

Öncesi: 1 gün gibi kısa bir sürede elime geçti. Ve bu fıyata süper bir ürün tavsiye ederim. Lakin eli büyük olan kişiler daha büyük modelini tercih edebilirler ortaboy ürün. Teşekkürler 
Sonrası: [  38   40   20  108    2  161   29  150    3    5 2855   73    2    4
    9   10 1159 2430  104  113 2068   11  104 1150  175    4   16]


Cümlenin iki durumunun boyutlarını incelediğimizde token dizisinin 3 kelime az olduğunu görürüz. Bunun sebebi **girdideki cümlede ilk 10 bin kelime arasına girememiş kelimeler** olmasıdır. Bu kelimeler çok nadir kullanıldığından işlenmesine gerek görülmüyor şimdilik.

In [18]:
print(len(x_train[IDX].split(" ")))
print(len(x_train_tokens[IDX]))

30
27


Girdilerin tamamının aynı boyutta olması bir gerçek. Ancak şu an tüm cümlelerin kendine ait boyutları var.

Bir sayı seçeceğiz ve verisetinde tüm cümlelerin kelime sayısı hemen hemen bu sayıya yakın veya ona eşit olacak.

In [19]:
total_sentences = x_train_tokens + x_test_tokens
num_tokens = np.array([len(tokens) for tokens in total_sentences])

In [20]:
np.mean(num_tokens)

20.744703220162876

Maksimum bir cümlenin kelime uzunluğu

Biraz matematik işlem ile en kullanışlı olacak kelime uzunluğunu buluyoruz.

In [21]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens) # np.std = standart sapma
max_tokens = int(max_tokens)
max_tokens

59

Cümlelerden kaçı 59 ve altı kelimeye sahipmiş görelim.

In [22]:
print("%", round(np.sum(num_tokens < max_tokens) / len(num_tokens) * 100, 2))

% 95.98


**PADDING**
- Cümlelerin her birinin 59 boyutunda olması gerektiğini bulduk. Peki bu sayıdan az olan veya çok uzunluğa sahip cümleler nasıl etkilenecek?

- 59 kelimeden fazla uzunluğu olan cümleler en baştan silinerek 59 kelimeye indirgenecek.
* 59 kelimeden az uzunluğu olan cümlelere ise başlarına 0 eklenerek 59'a tamamlanacak.

In [23]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad  = pad_sequences(x_test_tokens,  maxlen=max_tokens)

Artık verilerimiz 2 boyutlu bir numpy array olarak saklanıyor. Boyutlarını görelim.

In [24]:
x_train_pad.shape

(194797, 59)

In [25]:
x_test_pad.shape

(48700, 59)

Test cümle sayısının 194 bin, test sayısının 48 bin olduğunu da yeniden görmüş olduk.

Padding işleminin sonuçlarını örneklerle inceleyelim.

In [26]:
IDX = 800
print("{} \nUzunluk: {}\n".format(np.array(x_train_tokens[IDX]), len(x_train_tokens[IDX])))
print("{} \nUzunluk: {}\n".format(np.array(x_train_pad[IDX]), len(x_train_pad[IDX])))

[  19  341   52  194   30   54 7992   55  209  603 7887    3   36  114
  164  479   85    1 1682  782   30    2  380  326    3    1   81  132
  562    9   10] 
Uzunluk: 31

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
   19  341   52  194   30   54 7992   55  209  603 7887    3   36  114
  164  479   85    1 1682  782   30    2  380  326    3    1   81  132
  562    9   10] 
Uzunluk: 59



**Boyutu 31 olan bir cümle başına 0 verileri eklenerek 59'a tamamlanmış.**

In [27]:
IDX = 253
print("{} \nUzunluk: {}\n".format(np.array(x_train_tokens[IDX]), len(x_train_tokens[IDX])))
print("{} \nUzunluk: {}\n".format(np.array(x_train_pad[IDX]), len(x_train_pad[IDX])))

[   5   19  234   85 1223 3811 1410 1820  128   17   56  690 6254 1548
 2722 2254   71 3568  405 7548  112  894   71   34 1504  124  414 1738
    8  677   83   12    5   19  677 3010   46 1501   36   98   98   12
    5   19  146  307 1944 7548 5645    1    6   54   35  124  162  103
  791 1419   66   10    5   19   57 3630  113  125   46  176   57 1696] 
Uzunluk: 70

[ 690 6254 1548 2722 2254   71 3568  405 7548  112  894   71   34 1504
  124  414 1738    8  677   83   12    5   19  677 3010   46 1501   36
   98   98   12    5   19  146  307 1944 7548 5645    1    6   54   35
  124  162  103  791 1419   66   10    5   19   57 3630  113  125   46
  176   57 1696] 
Uzunluk: 59



**Boyutu 70 olan bir cümle başından gerekli kadar kelime silinerek 59'a tamamlanmış.**

Elimizde artık kelimeler yok onların token değerleri var. Token değerlerinden kelimelere gidebileceğimiz bir fonksiyon yazalım.

In [28]:
INVERSE_MAP = dict(zip(tokenizer.word_index.values(), tokenizer.word_index.keys()))

def tokens_to_words(sentence, tokens):
    words = [INVERSE_MAP[token] for token in tokens if token!=0]
    text = ' '.join(words)
    print("Öncesi: {} \n\nSonrası: {}".format(sentence, text))

In [29]:
IDX = 750
tokens_to_words(sentence=x_train[IDX], tokens=x_train_tokens[IDX])

Öncesi: bloetooth aparatı küçük ve şık. mouse ise çok kullanışlı.  bağlantı sorunu asla yok. aldım ve 1 yıldır kullanıyprum..almak isteyenleree siddetle tavsiya ederim. ucuz ve kaliteli bir ürün. 

Sonrası: aparatı küçük ve şık mouse ise çok kullanışlı bağlantı sorunu asla yok aldım ve 1 yıldır almak siddetle ederim ucuz ve kaliteli bir ürün


- yukarıda ilk olarak verisetinden bir yorum gördük, bazı kelimeler yanlış yazılmış.
* ardından ise tokenleştirme işleminden sonra ağımıza vereceğimiz cümlenin yeni halini gördük.
- yeni cümlede yanlış yazılan kelimeler veya az kullanılan kelimeler silinmiş.

## 4- RNN Oluşturma

Önişleme süreci bittikten sonra artık sinir ağını oluşturabiliriz.

In [45]:
have_a_model = True
MODEL_PATH = "../models/my_model.h5"

if have_a_model:
    model = load_model(MODEL_PATH)
    print("Found a model!")

else:
    print("Found no model. Creating one.")
    model = Sequential()

    embedding_size = 50

    model.add(Embedding(input_dim=num_words,
                        output_dim=embedding_size,
                        input_length=max_tokens,
                        name='embedding_layer'))

    model.add(GRU(units=16, return_sequences=True))
    model.add(GRU(units=8, return_sequences=True))
    model.add(GRU(units=4))
    model.add(Dense(1, activation='sigmoid'))

    optimizer = Adam(lr=1e-3)

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])

    model.fit(x_train_pad, y_train, epochs=5, batch_size=256)
    model.save(MODEL_PATH)
    clear_output()
    print("Model saved.")

Found a model!


Modelin özetini görelim.

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 59, 50)            500000    
_________________________________________________________________
gru_6 (GRU)                  (None, 59, 16)            3216      
_________________________________________________________________
gru_7 (GRU)                  (None, 59, 8)             600       
_________________________________________________________________
gru_8 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 5         
Total params: 503,977
Trainable params: 503,977
Non-trainable params: 0
_________________________________________________________________


Model eğitildi. Şimdi test için ayırdığımız veriyi modele verip çıktıyı görelim.

## 5- Analiz

In [32]:
result = model.evaluate(x_test_pad, y_test)

48700/48700 [==============================] - 35s 715us/sample - loss: 0.1576 - acc: 0.9530


Modelin %95 oranında başarılı olduğunu gördük.

In [33]:
result

[0.15759055556733262, 0.9529979]

In [34]:
num_true_sentence = int(len(x_test) * result[1])
print("Test verisindeki {} adet cümleden {} tanesi doğru bilindi.".format(len(x_test), num_true_sentence))

Test verisindeki 48700 adet cümleden 46410 tanesi doğru bilindi.


**Evaluate** fonksiyonu yalnızca accuracy ve loss değerini döndürür. Tek tek cümlelerin sonuçlarını görmek için **predict**'i kullanabiliriz.

In [35]:
y_pred = model.predict(x_test_pad)

In [36]:
y_pred.shape

(48700, 1)

Her cümle için çıktı 0 ile 1 arasındadır. 0 bildiğimiz gibi olumsuz 1 olumlu anlamındadır. 0.5 üzerini olumlu altını olumsuz olarak işaretleyelim.

In [37]:
test_pred = np.array([1 if p>0.5 else 0 for p in y_pred])

* Cümleyi
- Cümlenin Asıl Etiketini
* Cümlenin Üretilmiş Etikeni

bir arada görelim.

In [38]:
IDX = 1000
sentence = x_test[IDX]
real_rate = y_test[IDX]
predicted_rate = test_pred[IDX]

print("Cümle: {} \nAsıl Etiket: {} \nÜretilen Etiket: {}".format(sentence, real_rate, predicted_rate))

Cümle: işyerindeki yavru kedilerimiz için aldım, bayılıyorlar yerken. Başka bir mamaya alışmamış yavrular için bence uygun bir seçenek. Fiyatı da diğerlerinden uygun. 
Asıl Etiket: 1 
Üretilen Etiket: 1


## 6- Testler

Şimdi kendi yazdığımız birkaç cümle ve sonuçlarına göz atalım.

Test cümlelerini tanımlayalım.

In [39]:
sentence = ["iyi paketleme ama beğenmedim.",
            "Çok kötü.",
            "Kötü kargo ürün kötü değil ama kullanım tarihi geçmiş."
           ]

Tokenleştirelim.

In [40]:
tokens = tokenizer.texts_to_sequences(sentence)

Padding işlemini gerçekleştirelim.

In [41]:
tokens_pad = pad_sequences(tokens, maxlen=59)

Test token'larını modele verelim.

In [42]:
results = model.predict(tokens_pad)

Sonuçlardan 0.5 üzerini olumlu, diğerlerini olumsuz yapıp çıktıyı anlamlaştıralım.

In [43]:
new_pred = np.array([1 if result>0.5 else 0 for result in results])

In [44]:
for s,r in zip(sentence, new_pred):
    see_example(s,r)

Cümle: iyi paketleme ama beğenmedim. 
Etiket: Olumsuz
Cümle: Çok kötü. 
Etiket: Olumsuz
Cümle: Kötü kargo ürün kötü değil ama kullanım tarihi geçmiş. 
Etiket: Olumlu
